In [ ]:
# import os,zipfile
# os.chdir('drive/My Drive/twitter-bot-detection')
# !pwd

In [ ]:
''' Download datasets ,unzip them and store to drive '''
# !mkdir datasets
# os.chdir('datasets')
# !wget https://botometer.osome.iu.edu/bot-repository/datasets/cresci-2017/cresci-2017.csv.zip
# !wget http://infolab.tamu.edu/data/social_honeypot_icwsm_2011.zip 
# !mkdir cresci-2017
# !unzip -q cresci-2017.csv.zip -d cresci-2017
# !unzip -q social_honeypot_icwsm_2011.zip 
# !rm cresci-2017.csv.zip
# !rm social_honeypot_icwsm_2011.zip
# os.chdir('..')

' Download datasets ,unzip them and store to drive '

In [ ]:
# # unzip files in cresci-2017
# dir_name = 'datasets/cresci-2017/datasets_full.csv'
# extension = ".zip"
# os.chdir(dir_name) # change directory from working dir to dir with files

# for item in os.listdir(): # loop through items in dir
#     if item.endswith(extension):
#       zip_ref = zipfile.ZipFile(item) # create zipfile object
#       zip_ref.extractall() # extract file to dir
#       zip_ref.close() # close file
#       os.remove(item) # delete zipped file

In [ ]:
import sys
import os
import tweepy
import numpy as np
import pandas as pd
import json
from datetime import datetime, timezone
import re
import time

In [ ]:
os.chdir('drive/My Drive/twitter-bot-detection')
os.getcwd()

'/content/drive/My Drive/twitter-bot-detection'

In [ ]:
# cresci-2017 dataset
# Genuine users
assert(os.getcwd() == '/content/drive/My Drive/twitter-bot-detection' )

gu_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/genuine_accounts.csv/users.csv', sep = ',')
gu_list = gu_df['id'].values.astype(int)

# Social spambots
ssbots1_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/social_spambots_1.csv/users.csv', sep = ',')
ssbots1_list = ssbots1_df['id'].values.astype(int)
ssbots2_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/social_spambots_2.csv/users.csv', sep = ',')
ssbots2_list = ssbots2_df['id'].values.astype(int)
ssbots3_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/social_spambots_3.csv/users.csv', sep = ',')
ssbots3_list = ssbots3_df['id'].values.astype(int)

# traditional spambots
tsbots1_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/traditional_spambots_1.csv/users.csv', sep = ',')
tsbots1_list = tsbots1_df['id'].values.astype(int)
tsbots2_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/traditional_spambots_2.csv/users.csv', sep = ',')
tsbots2_list = tsbots2_df['id'].values.astype(int)
tsbots3_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/traditional_spambots_3.csv/users.csv', sep = ',')
tsbots3_list = tsbots3_df['id'].values.astype(int)
tsbots4_df = pd.read_csv('./datasets/cresci-2017/datasets_full.csv/traditional_spambots_4.csv/users.csv', sep = ',')
tsbots4_list = tsbots4_df['id'].values.astype(int)

# combine social spambots and traditional spambots
ssbots_list = list(ssbots1_list) + list(ssbots2_list) + list(ssbots3_list)
tsbots_list = list(tsbots1_list) + list(tsbots2_list) + list(tsbots3_list) + list(tsbots4_list)

In [ ]:
#Social honeypot dataset

# Legitimate user info
lu_df = pd.read_csv('./datasets/social_honeypot_icwsm_2011/legitimate_users.txt', sep = '\t', header = None)
lu_df.columns = ['UserID', 'CreatedAt', 'CollectedAt', 'NumerOfFollowings', 'NumberOfFollowers', 'NumberOfTweets', 'LengthOfScreenName', 'LengthOfDescriptionInUserProfile']

# Content polluters info
bots_df = pd.read_csv('./datasets/social_honeypot_icwsm_2011/content_polluters.txt', sep = '\t', header = None)
bots_df.columns = ['UserID', 'CreatedAt', 'CollectedAt', 'NumerOfFollowings', 'NumberOfFollowers', 'NumberOfTweets', 'LengthOfScreenName', 'LengthOfDescriptionInUserProfile']

# Construct user id lists
lu_list = lu_df['UserID'].values.astype(int)
bot_list = bots_df['UserID'].values.astype(int)

In [ ]:
# scrape data from tweepy api
auth = tweepy.OAuthHandler('NorVL10UJjR7tgb9ORBHuFohm', '77RjNc1BxLfJXp0SV6LhUweLoP2mxID96KHlCN7n7lC2GYxGuv')

auth.set_access_token('764321884041310208-UwgcHEi3szzvkWKJXzaxT3c8qUC5Oo7', 'IRyiEyKvpN4ZfIVwAlTfpOqk2f1CFfEFSEjiayV5BZ2na')

api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:

# Given a name list and number of tweets needed to extract for each account
# Return a dictionary of dataframes
# Each dataframe contains info of one user
def API_scrap(name_list, count_num):
    fail_list = []
    user_dfs = {}
    count = 0
    for name in name_list:
        if count%300 == 0:
            print(count)
        count += 1
        try:
            data = api.user_timeline(name, count = count_num, tweet_mode = 'extended')
            user_dfs[name] = pd.DataFrame()
            for i in range(len(data)):
                jdata = pd.json_normalize(data[i]._json)
                user_dfs[name] = user_dfs[name].append(jdata, ignore_index=True)

        except :
            fail_list.append(name)
            continue
    
    return user_dfs, fail_list



In [ ]:
# sh_user_dfs, sh_user_fail_lst = API_scrap(lu_list, 10)
# sh_bot_dfs, sh_bot_fail_lst = API_scrap(bot_list[22224:], 10)
# gu_dfs, fail_list = API_scrap(gu_list, 10)
# ssbots_dfs, ssbots_fail_lst = API_scrap(ssbots_list, 10)
# tsbots_dfs, tsbots_fail_lst = API_scrap(tsbots_list, 10)

0
300
600


Rate limit reached. Sleeping for: 682


900
1200
1500


Rate limit reached. Sleeping for: 594


1800
2100
2400


...

2700
3000
3300


Rate limit reached. Sleeping for: 588


3600
3900
4200


Rate limit reached. Sleeping for: 586


4500
4800


In [ ]:
def user_id(df):
    try:
        return df['user.id_str'][0]
    except:
        return None
    
# Screen name length
def sname_len(df):
    try:
        return len(df['user.screen_name'][0])
    except:
        return None

# Number of digits in screen name
def sname_digits(df):
    try:
        return sum(c.isdigit() for c in df['user.screen_name'][0])
    except:
        return None
    
# User name length
def name_len(df):
    try: 
        return len(df['user.name'][0])
    except:
        return None

# Default profile
def def_profile(df):
    try:
        return int(df['user.default_profile'][0]*1)
    except:
        return None

# Default picture
def def_picture(df):
    try:
        return int(df['user.default_profile_image'][0]*1)
    except:
        return None

# Account age (in days)
def acc_age(df):
    try:
        d0 = datetime.strptime(df['user.created_at'][0],'%a %b %d %H:%M:%S %z %Y')
        d1 = datetime.now(timezone.utc)
        return (d1-d0).days
    except:
        return None

# Number of unique profile descriptions
def num_descrip(df):
    try:
        string = df['user.description'][0]
        return len(re.sub(r'\s', '', string).split(','))
    except:
        return None

# Number of friends
def friends(df):
    try:
        return df['user.friends_count'][0]
    except: 
        return None

# Number of followers
def followers(df):
    try:
        return df['user.followers_count'][0]
    except: 
        return None

# Number of favorites
def favorites(df):
    try:
        return df['user.favourites_count'][0]
    except:
        return None

# Number of tweets (including retweets, per hour and total)
def num_tweets(df):
    try:
        total = df['user.statuses_count'][0]
        per_hour = total/(acc_age(df)*24)
        return total, per_hour
    except:
        return None, None

def tweets_time(df):
    try:
        time_lst = []
        for i in range(len(df)-1):
            if df['retweeted'][i] == False:
                time_lst.append(df['created_at'][i])

        interval_lst = []
        for j in range(len(time_lst)-1):
            d1 = datetime.strptime(df['created_at'][j],'%a %b %d %H:%M:%S %z %Y')
            d2 = datetime.strptime(df['created_at'][j+1],'%a %b %d %H:%M:%S %z %Y')
            interval_lst.append((d2-d1).seconds)

        return np.array(interval_lst)
    except:
        return None

In [ ]:
# Given a dictionary of dataframes with one dataframe for each user
# this function processes the data and extracted all the user-related features
# and saves it to a dataframe with one row for each user

def create_df(user_dfs, filename):
    columns_lst = ['User ID', 'Screen name length', 'Number of digits in screen name', 'User name length', 'Default profile (binary)','Default picture (binary)','Account age (days)', 'Number of unique profile descriptions','Number of friends','Number of followers','Number of favorites','Number of tweets per hour', 'Number of tweets total','timing_tweet']

    # Create user dataframe
    user_full_df = pd.DataFrame(columns = columns_lst)
    count = 0
    for name in user_dfs.keys():
        df = user_dfs[name]
        tweets_total, tweets_per_hour = num_tweets(df)
        data = [user_id(df), sname_len(df), sname_digits(df), name_len(df), def_profile(df), def_picture(df), acc_age(df), num_descrip(df), friends(df), followers(df), favorites(df), tweets_per_hour, tweets_total, np.mean(tweets_time(df))]
        user_full_df.loc[count] = data
        count += 1

    user_full_df = user_full_df.dropna()
    # user_full_df.to_csv(filename+'.csv', encoding='utf-8', index=False)
    return user_full_df


In [ ]:
gu_full_df = create_df(gu_dfs, 'gu_dataframe')
ssbots_full_df = create_df(ssbots_dfs, 'ssbots_dataframe')
tsbots_full_df = create_df(tsbots_dfs, 'tsbots_dataframe')
sh_user_full_df = create_df(sh_user_dfs, 'sh_user_dataframe')
sh_bots_full_df = create_df(sh_bot_dfs, 'sh_bot_dataframe')

In [ ]:
c = pd.read_csv('data/failed/sh_bot_fail_lst')
clist = c['0'].values.astype(int)
clist = clist[2500:7500]
a,b = API_scrap(clist, 10)
t = create_df(a, 'a')
comb = pd.concat([t,bot_sh])
comb.to_csv('data/sh_bot_dataframe.csv', encoding='utf-8', index=False)
# failed list checked status :
#  tsbots - incomplete
#  sh bots - complete till 7500
#  sh user - complete till 15000

In [ ]:
#combine bots and users
bot_sh = pd.read_csv('data/sh_bot_dataframe.csv')
bot_ss = pd.read_csv('data/ssbots_dataframe.csv')
bot_ts = pd.read_csv('data/tsbots_dataframe.csv')
final_bot = pd.concat([bot_sh,bot_ss,bot_ts])
user_sh = pd.read_csv('data/sh_user_dataframe.csv')
user_cresai = pd.read_csv('data/gu_dataframe.csv')
final_user = pd.concat([user_sh,user_cresai])

In [ ]:
for i in [bot_sh,bot_ss,bot_ts,final_bot]:
    print(i.shape)
print()
for i in [user_sh,user_cresai,final_user]:
    print(i.shape) 

(7173, 14)
(4135, 14)
(383, 14)
(11691, 14)

(11649, 14)
(2387, 14)
(14036, 14)


In [ ]:
final_user.head()

,User ID,Screen name length,Number of digits in screen name,User name length,Default profile (binary),Default picture (binary),Account age (days),Number of unique profile descriptions,Number of friends,Number of followers,Number of favorites,Number of tweets per hour,Number of tweets total,timing_tweet
0,54149901,13,0,18,1,0,4145,1,1,157,0,0.226186,22501,48255.000
1,54175862,12,0,12,0,0,4145,1,1582,1028,196,0.130941,13026,45240.375
2,54181293,9,0,9,0,0,4145,1,791,270,3,0.030338,3018,54783.250
3,54190543,15,1,22,0,0,4145,1,1565,3405,7328,0.133484,13279,71398.625
4,54193711,6,0,1,0,0,4145,1,1380,738,209,0.045245,4501,32269.500


In [ ]:
# export to csv
final_user.to_csv('data/final_users.csv', encoding='utf-8', index=False)
final_bot.to_csv('data/final_bots.csv', encoding='utf-8', index=False)